<a href="https://colab.research.google.com/github/panzershracker/Web-scraping/blob/master/func_tests4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import requests, re
import time as timer
from datetime import datetime, date, time, timedelta
from bs4 import BeautifulSoup as soup
from pprint import pprint



In [0]:
def get_vac_names(resp_text):
  
  div = str(resp_text.find_all('div', {'class': ['clearfix vacancy premium ',
                                                 'clearfix vacancy']}))

  h2 = soup(div, 'lxml').find_all('h2')

  vac_names = []

  for i in h2:

    vac_names.append(i.a.getText())

  return vac_names

In [0]:
def time_ago(resp_text):
  
  span = resp_text.find_all('span', {'class': 'date'})
  
  time_ago = []
  
  for i in range(0, len(span)):
    time = span[i].find(text=True, recursive=False).replace('  ', '')[1:]
    time_ago.append(time)
    
  return time_ago
  

In [0]:
def post_datetime(time_ago, resp_text):
  
  time_ago = time_ago(resp_text)  
 
  date_time = []
  
  for i in time_ago:
    
    now = datetime.now()
      
    if not i:
      date_time.append(0)
      
    else: 

      num, rest = i.split(' ', 1)

      if rest.startswith('ч'):
        date_time.append(now - timedelta(hours= int(num)))

      elif rest.startswith('д'):
        date_time.append(now - timedelta(days= int(num)))

      elif rest.startswith('н'):
        date_time.append(now - timedelta(weeks= int(num)))

      elif rest.startswith('м'):
        date_time.append(now - timedelta(weeks= int(num)*4))

      else:
        date_time.append(0)

    
  return date_time
  

In [0]:
def get_salaries(resp_text):
   
    h2 = resp_text.find_all('h2')
        
    salaries = []
    
    for i in h2:
      sal = re.findall(mask, i.b.getText())
      salaries.append(''.join(sal))
      
    return salaries

In [0]:
def get_links(resp_text):
  
  div = str(soup(resp.text, 'lxml').find_all('div', {'class': ['clearfix vacancy premium ',
                                                                 'clearfix vacancy']}))
  h2 = str(soup(div, 'lxml').find_all('h2'))
  
  a = list(soup(h2, 'lxml').find_all('a'))
  
  vac_links = []

  for i in range(0, len(a)):
    vac_links.append('http://gorodrabot.ru' + a[i].get('href'))
        

  return vac_links


# ! TODO 

### Переделать без множественного парсинга в функциях
(get_links, hirer, get_comp и тд, где возможно)

In [0]:
def hirer(resp_text):
  
  div = str(soup(resp.text, 'lxml').find_all('div', {'class': ['clearfix vacancy premium ',
                                                                 'clearfix vacancy']}))
  h2 = str(soup(div, 'lxml').find_all('h2'))
  
  a = list(soup(h2, 'lxml').find_all('a'))
  
  hirers = []

  for i in range(0, len(a)):
    hirers.append(a[i].get('data-action'))
        

  return hirers

In [0]:
# def get_comp():
  
#   div = str(soup(resp.text, 'lxml').find_all('div', {'class': ['clearfix vacancy premium ',
#                                                                'clearfix vacancy']}))
#   div = str(soup(div, 'lxml').find_all('div', {'class': 'address'}))
  
#   span = list(soup(div, 'lxml').find_all('span'))
  
#   comp = []
  
#   for i in range(0, len(span)):
#     comp.append(span[i])

#   return comp

# ! TODO

### Доделать вывод названия компаний.

In [0]:
def salary_separator(resp_text):
  
  ot = []
  do = []
  
  salaries = get_salaries(resp_text)
  
  for i in salaries:
    
    if i.startswith('от'):
      ot.append(int(i[2:]))
      do.append(0)
    
    elif i.startswith('до'):
      
      if i.startswith('дог'):        
        ot.append(-1)
        do.append(-1)
      
      else:        
        do.append(int(i[2:]))
        ot.append(0)
        
#     elif i[0].isnumeric():      
#       ot.append(int(i))
#       do.append(int(i))
            
    else:      
      a = i.split('—')
      
      ot.append(int(a[0]))
      do.append(int(a[1]))
      
  return ot, do

In [0]:
def temp_df(time_ago, resp_text):
  
  df = pd.DataFrame({'vacancy': get_vac_names(resp_text),
                   'salary min': salary_separator(resp_text)[0],
                   'salary max': salary_separator(resp_text)[1],
                   'link': get_links(resp_text),
                   'from': hirer(resp_text),
                   'datetime': post_datetime(time_ago, resp_text)})
  
  return df

In [0]:
mask = '[а-я0-9—]+'
vacancy = 'программист'
language = 'python'
city = 'moskva.'
salary = '?st=1'

In [0]:
result = pd.DataFrame(columns=['vacancy',
                               'salary min',
                               'salary max',
                               'link',
                               'from',
                               'datetime'])

In [308]:
with requests.Session() as session:
  
    for i in range(1, 5):
      
      page = f'&p={i}'

      session.headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'

      url = f'https://{city}gorodrabot.ru/{vacancy}_{language}{salary}{page}'
      resp = session.get(url)
      
      timer.sleep(10)

      resp_text = soup(resp.text, 'lxml')

      print(f'gorodrabot.ru status code is: {resp.status_code}\n'
            f'Обработка страницы {i}\n')

      df = temp_df(time_ago, resp_text)
      
      result = result.append(df, ignore_index=True)

gorodrabot.ru status code is: 200
Обработка страницы 1

gorodrabot.ru status code is: 200
Обработка страницы 2

gorodrabot.ru status code is: 200
Обработка страницы 3

gorodrabot.ru status code is: 200
Обработка страницы 4



In [309]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 6 columns):
vacancy       107 non-null object
salary min    107 non-null object
salary max    107 non-null object
link          107 non-null object
from          106 non-null object
datetime      107 non-null object
dtypes: object(6)
memory usage: 5.1+ KB


In [310]:
result.head(10)

,vacancy,salary min,salary max,link,from,datetime
0,Программист php,50000,70000,http://gorodrabot.ru/vacancy/198375260/program...,rabota.ru,2019-08-06 13:10:38.512665
1,Frontend разработчик,120000,0,http://gorodrabot.ru/vacancy/199750234/fronten...,rabota.ru,2019-08-14 13:10:38.512709
2,Backend-разработчик (python 3),100000,200000,http://gorodrabot.ru/vacancy/193786676/backend...,talents.yandex.ru,2019-08-20 05:10:38.512713
3,Windows-разработчик со знанием JS,80000,0,http://gorodrabot.ru/vacancy/198424878/windows...,None,0
4,Intelligent R&D Tools Technical Expert,300000,0,http://gorodrabot.ru/vacancy/196539228/intelli...,kellyservices.ru,2019-07-30 13:10:38.512720
5,Разработчик Python,100000,180000,http://gorodrabot.ru/vacancy/199205772/razrabo...,hh.ru,2019-08-19 14:10:38.512724
6,Разработчик Python Backend,150000,0,http://gorodrabot.ru/vacancy/200375565/razrabo...,hh.ru,2019-08-19 13:10:38.512727
7,Программист Python,90000,0,http://gorodrabot.ru/vacancy/197245059/program...,hh.ru,2019-08-19 13:10:38.512730
8,Junior Python developer / Начинающий разработч...,60000,70000,http://gorodrabot.ru/vacancy/200153543/junior_...,hh.ru,2019-08-19 13:10:38.512732
9,Backend-разработчик python,130000,0,http://gorodrabot.ru/vacancy/200407757/backend...,hh.ru,2019-08-19 18:10:38.512735


In [0]:
# print(len(get_vac_names(resp_text)))
# print(len(get_salaries(resp_text)))
# print(len(get_links(resp_text)))
# print(len(salary_separator(resp_text)[0]))
# print(len(salary_separator(resp_text)[1]))
# print(len(post_datetime(time_ago, resp_text)))
# print(len(hirer(resp_text)))